In [1]:
from dotenv import load_dotenv
import os
from supabase import create_client
import fonctions.fonctions_api as f
import fonctions.fonctions_tableaux as ft
import fonctions.fonctions_standard as fs

C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()  # charge le .env
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
season = os.getenv("SEASON")
supabase = create_client(url, key)

In [7]:
res = supabase.table("vue_tableau_recap") \
    .select("id_contrat") \
    .execute()

# Extraire les identifiants uniques
id_contrats = list({row["id_contrat"] for row in res.data}) if res.data else []


INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/vue_tableau_recap?select=id_contrat "HTTP/2 200 OK"


In [ ]:
perf_res = supabase.table("Performance") \
    .select("PER, id_match") \
    .eq("id_contrat", id_contrat) \
    .execute()

1000

In [8]:
id_contrats

[515,
 516,
 519,
 520,
 522,
 528,
 533,
 534,
 545,
 555,
 559,
 571,
 572,
 574,
 575,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 330,
 331,
 332,
 333,
 334,
 335,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 452,
 453,
 488,
 489,
 490,
 491,
 492,
 493,
 495,
 497,
 498,
 499,
 500,
 503,
 504,
 506]

In [5]:
for idc in id_contrats :
    ft.create_pnj_perf(supabase,idc)

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/vue_tableau_recap?select=id_contrat%2Cdate%2CPER%2Cnom%2Cprenom%2Cnom_equipe%2Crang%2Cid_joueur&id_contrat=eq.515 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/vue_tableau_recap?select=id_contrat%2Cdate%2CPER%2Cnom%2Cprenom%2Cnom_equipe%2Crang%2Cid_joueur&id_contrat=eq.516 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/vue_tableau_recap?select=id_contrat%2Cdate%2CPER%2Cnom%2Cprenom%2Cnom_equipe%2Crang%2Cid_joueur&id_contrat=eq.519 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/vue_tableau_recap?select=id_contrat%2Cdate%2CPER%2Cnom%2Cprenom%2Cnom_equipe%2Crang%2Cid_joueur&id_contrat=eq.520 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/vue_tableau_recap?select=id_contrat%2Cdate%2CPER%2Cnom%2Cprenom%2Cnom_equipe%2Crang%2Cid_j

In [ ]:
f.acheter_joueur(supabase, id_user=4, id_contrat=318)

In [ ]:
ft.afficher_effectif(supabase,id_user=4)

In [ ]:
ft.afficher_joueurs_disponibles(supabase,id_user=4)

In [ ]:
for id_contrat in [329,441,494,496,540,541,546,570]:
    f.finir_contrat_with_idcontrat(supabase,id_contrat=id_contrat)

In [ ]:
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 323)
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 360)
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 435)
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 328)

In [ ]:
import bcrypt
import fonctions.fonctions_standard as fs

plain_password = "mdp"
hashed = fs.hash_password(plain_password)

# Vérification
bcrypt.checkpw(plain_password.encode(), hashed.encode())  # → True


In [ ]:
fs.hash_password("mdp") == fs.hash_password("mdp")

In [ ]:
# f.ajouter_user(supabase, pseudo = "gpain1999", mot_de_passe = "Iledefrance75@", adresse_mail = "guillaume.pain9@gmail.com")

In [ ]:
# f.finir_contrats_equipe(supabase, id_equipe = "ZAL")

In [ ]:
from euroleague_api.game_stats import GameStats

gs = GameStats()

ggs = gs.get_game_report(season=season, game_code=324)

In [ ]:
# f.get_update_match_data(supabase, season)